In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
%reload_ext autoreload

In [21]:
import logging
import logging.config
import configparser
import os
import time
from pathlib import Path
import signal

import lmsquery
from waveshare_epd import epd5in83

In [8]:
import cfg
from epdlib import *

In [9]:
# this works best as a global variable
logConfig = Path(cfg.LOGCONFIG)
logging.config.fileConfig(logConfig.absolute())
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(name)s %(levelname)s: %(message)s')

logger = logging.getLogger(__name__)

In [11]:
def configuration(configFile=None):
    
    configDefaults = cfg.CONFIGDEFAULTS
    
    if not configFile:
         configFile = Path(cfg.CONFIGFILE)

    config = configparser.ConfigParser()
    logger.info(f'reading configuration: {configFile}')
    config.read(configFile)
    
    try:
        for section in configDefaults:
            if section not in config.sections():
                logger.debug(f'adding section: {section}')
                config.add_section(section)
            
            for option in configDefaults[section]:
                if not config.has_option(section, option):
                    logger.debug(f'missing option: {option}')
                    logger.debug(f'setting {option} to: {configDefaults[section][option]}')
                    config[section][option] = str(configDefaults[section][option])
            with open(configFile, 'w') as file:
                config.write(file)
    
    except Exception as e:
        logging.exception(f'exception on configuration file: {e}')
        raise
    
    logger.debug(f'config file contains sections: {config.sections()}')
    return config

In [15]:
def query(lms, last=0, delay=7):
    '''query the player only when a specified delay has passed
    Accepts:
        last: float - last time query was called

    Returns:
        tuple(last, lms.now_playing()
    '''
    if last==0:
        last = time.clock_gettime(time.CLOCK_MONOTONIC)-delay

    if time.clock_gettime(time.CLOCK_MONOTONIC) > last+delay:
        return time.clock_gettime(time.CLOCK_MONOTONIC), lms.now_playing()

    else:
        return last, None

In [16]:
class signalHandler(object):
    '''handle specific signals and allow graceful exiting while loop
    https://stackoverflow.com/a/31464349/5530152
    
    Signals Handled Gracefully:
        SIGINT
        SIGTERM
    Atributes:
        kill_now (bool) default: False
    '''
    kill_now = False
    def __init__(self):
        signal.signal(signal.SIGINT, self.exit_gracefully)
        signal.signal(signal.SIGTERM, self.exit_gracefully)
    
    def exit_gracefully(self, signum, frame):
        self.kill_now = True

In [22]:
def main():
    '''main entry point
    '''   
    logger.setLevel(logging.DEBUG)
    logger.info('Starting program')
    config = configuration()
    # scan for lms server and use the first one (this may be a terrible idea for other people)
#     lmsServer = lmsquery.scanLMS()[0]
    
    # define LMS query object
#     lms = lmsquery.LMSQuery(lmsServer['host'], lmsServer['port'], config.get('server', 'player'))
    lms = lmsquery.LMSQuery(player_id=config.get('server', 'player'))
    
    
    sigHandler = signalHandler()
    
    # last update
    updated = 0
    # id of currently playing track
    nowPlayingID = None
    # status of player 
    nowPlayingMode = None

    # now playing fields to display
    fields = ['title', 'album', 'artist', 'mode']
    
    
    print(f'pid: {os.getpid()}')
    try:
        while not sigHandler.kill_now:
            updated, value = query(lms, updated)
            response = value
            if response:
                if response['id'] != nowPlayingID or response['mode'] != nowPlayingMode:
                    nowPlayingID = response['id']
                    nowPlayingMode = response['mode']
                    for field in fields:
                        print(f'{field}: {response[field]}')
                    print('\n')
            time.sleep(0.5)
    finally:
        print("cleaning up")
    

In [ ]:
if __name__ == '__main__':
    
    foo = main()

__main__         5:     INFO - Starting program
__main__         9:     INFO - reading configuration: servercfg.ini
__main__         30:     DEBUG - config file contains sections: ['server', 'screen']
pid: 1553
title: Ease Your Feet in the Sea
album: The Boy With the Arab Strap
artist: Belle and Sebastian
mode: stop


title: Diplomat's Son
album: Contra
artist: Vampire Weekend
mode: play


title: Diplomat's Son
album: Contra
artist: Vampire Weekend
mode: pause


title: #607: Didn't We Solve This One?
album: This American Life
artist: This American Life
mode: play


title: #607: Didn't We Solve This One?
album: This American Life
artist: This American Life
mode: pause


title: Diplomat's Son
album: Contra
artist: Vampire Weekend
mode: play


title: Diplomat's Son
album: Contra
artist: Vampire Weekend
mode: pause


title: California English
album: Contra
artist: Vampire Weekend
mode: play


title: Diplomat's Son
album: Contra
artist: Vampire Weekend
mode: play




In [ ]:
l = Layout(layout=layouts.threeRow)
l.set_images()

In [ ]:
uA = {'title': "Steam Engine",
      'artist': "My Morning Jacket",
      'album': "It Still Moves",
      'status': "Playing",
      'coverart': "./cover.jpg"}
uB = {'title': "This Tornado Loves You",
      'artist': "Neko Case",
      'album': "Middle Cyclone",
      'status': "Paused",
      'coverart': "./ozo_cover.jpg"}


In [ ]:
e = epd5in83.EPD()
s = Screen()
s.epd = e
s.initEPD()

In [ ]:
l.update_contents(uA)

In [ ]:
s.elements=l.blocks.values()
s.concat()
s.writeEPD()

In [ ]:
s.initEPD()
s.clearEPD()